In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os, random

print(tf.__version__)

BATCH_SIZE = 32
IMG_SIZE = (300, 300)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

ImportError: cannot import name 'device_lib' from 'tensorflow.python.client' (unknown location)

In [2]:
CLASS = ('E', 'I')
TARGET_PATH = lambda x : f"data\\images\\{x}\\{CLASS[0]}_{CLASS[1]}"

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(TARGET_PATH("train"),
                                                                    shuffle=True,
                                                                    batch_size=BATCH_SIZE,
                                                                    image_size=IMG_SIZE,
                                                                    )

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(TARGET_PATH("test"),
                                                                        shuffle=True,
                                                                        batch_size=BATCH_SIZE,
                                                                        image_size=IMG_SIZE,
                                                                        )
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 2)
validation_dataset = validation_dataset.skip(val_batches // 2)

AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Rescal pixel value between [0, 255] to [-1, 1]
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1. / 127.5, offset=-1)
IMG_SHAPE = IMG_SIZE + (3,)

Found 16587 files belonging to 2 classes.
Using 11611 files for training.
Found 16587 files belonging to 2 classes.
Using 4976 files for validation.


In [3]:
def plot_accuracy_loss(acc, val_acc, loss, val_loss):
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label= 'Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()), 1])
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0, 1.0])
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

In [8]:
n_input = 20
n_hidden_1 = 20
n_hidden_2 = 20
n_hidden_3 = 20
n_hidden_4 = 20
n_hidden_5 = 20
n_hidden_6 = 20
n_output = 1

base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE, include_top=False)
base_model.trainable = False

MPL_model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=n_hidden_1), # dense는 activation 주로 안 씀 
    tf.keras.layers.Dense(units=n_hidden_2),
    tf.keras.layers.Dense(units=n_hidden_3),
    tf.keras.layers.Dense(units=n_hidden_4),
    tf.keras.layers.Dense(units=n_hidden_5),
    tf.keras.layers.Dense(units=n_hidden_6),
    tf.keras.layers.Dense(units=n_output),
])

inputs = tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = preprocess_input(inputs)
x = base_model(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(units = 128)(x)
outputs = MPL_model(x)
model = tf.keras.Model(inputs, outputs)

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), # Softmax activation function applied
              metrics=['accuracy'])

total_epochs=20
model.fit(train_dataset,
          epochs=total_epochs,
          validation_data=validation_dataset)

acc = model.history['accuracy']
val_acc = model.history['val_accuracy']
loss = model.history['loss']
val_loss = model.history['val_loss']

plot_accuracy_loss(acc, val_acc, loss, val_loss)

# Evaluation of model
loss, accuracy = model.evaluate(test_dataset) # accuracy 는 0, 1 
print('Test accuracy :', accuracy)
# loss 는 확률 차이, accuracy는 0, 1차이


Epoch 1/100
363/363 [==============================] - 397s 1s/step - loss: 0.7014 - accuracy: 0.5070 - val_loss: 0.6886 - val_accuracy: 0.5165
Epoch 2/100
363/363 [==============================] - 377s 1s/step - loss: 0.6889 - accuracy: 0.5137 - val_loss: 0.6958 - val_accuracy: 0.5302
Epoch 3/100
363/363 [==============================] - 380s 1s/step - loss: 0.6860 - accuracy: 0.5149 - val_loss: 0.6868 - val_accuracy: 0.5343
Epoch 4/100
363/363 [==============================] - 380s 1s/step - loss: 0.6833 - accuracy: 0.5172 - val_loss: 0.6835 - val_accuracy: 0.5282
Epoch 5/100
363/363 [==============================] - 384s 1s/step - loss: 0.6808 - accuracy: 0.5228 - val_loss: 0.6829 - val_accuracy: 0.5335
Epoch 6/100
363/363 [==============================] - 386s 1s/step - loss: 0.6776 - accuracy: 0.5283 - val_loss: 0.7035 - val_accuracy: 0.5520
Epoch 7/100
363/363 [==============================] - 384s 1s/step - loss: 0.6797 - accuracy: 0.5367 - val_loss: 0.6852 - val_accuracy:

TypeError: 'History' object is not subscriptable